# Imports

In [ ]:
# Add local repository to python path -> allows for importing here
import sys
sys.path.append(##FIXME##'/COMBINE')
sys.path.append(##FIXME##'/MUSCL')

In [ ]:
import numpy as np
import xarray as xr
import torch

# OGGM models
from oggm.core.massbalance import LinearMassBalance as oggm_MassBalance
from oggm.core.flowline import FluxBasedModel as oggm_FluxModel
from oggm.core.flowline import RectangularBedFlowline as oggm_rect_flowline
from oggm.core.flowline import ParabolicBedFlowline as oggm_para_flowline
from oggm.core.flowline import TrapezoidalBedFlowline as oggm_trap_flowline

# MUSCLsuperbee model
from MUSCL.MUSCL_flowline import MUSCLSuperBeeModel

# COMBINE models
from combine1d.core.flowline_adapted import FluxBasedModel as combine_FluxModel
from combine1d.core.flowline_adapted import RectangularBedFlowline as combine_rect_flowline
from combine1d.core.flowline_adapted import ParabolicBedFlowline as combine_para_flowline
from combine1d.core.flowline_adapted import TrapezoidalBedFlowline as combine_trap_flowline

# topography function
from combine1d.sandbox.idealized_experiments import define_geometry, define_mb_model

# other stuff
from combine1d.core.type_conversions import to_numpy_array
import oggm.cfg as cfg
cfg.initialize()

# Define experiments

- equilibrium: ELA = 3300 
- advancing: ELA1 = 3300  ELA2 = 3100
- retreating: ELA1 = 3100, ELA2 = 3300
- define times after oggm run

# Function for computation

In [ ]:
different_bed_h = ['linear', 'cliff', 'random']
different_widths = ['constant', 'wide_top']
different_bed_shapes = ['rectangular', 'parabolic', 'trapezoidal']
different_glacier_states = ['equilibrium', 'advancing', 'retreating']
different_models = ['OGGM', 'COMBINE', 'MUSCL']

results = {}
for shape in different_bed_shapes:
    results[shape] = {}
    for bed_h in different_bed_h:
        results[shape][bed_h] = {}
        for width in different_widths:
            results[shape][bed_h][width] = {}
            for state in different_glacier_states:
                results[shape][bed_h][width][state] = {}
                for model in different_models:
                    results[shape][bed_h][width][state][model] = {}

# create array of all possibilities for reader frindly for loop
all_possibilities = []
for shape in different_bed_shapes:
    for bed_h in different_bed_h:
        for width in different_widths:
            for state in different_glacier_states:
                all_possibilities.append((shape, bed_h, width, state))
                
                
used_bed_shape = different_bed_shapes[0]
used_bed_h = different_bed_h[0]
used_widths = different_widths[0]
used_glacier_state = different_glacier_states[0]

for used_bed_shape, used_bed_h, used_widths, used_glacier_state in all_possibilities:
    # get geometry
    geometry = define_geometry(used_bed_h_geometry=used_bed_h,
                               used_along_glacier_geometry=used_widths,
                               bed_geometry=used_bed_shape)

    # variable to decide if muscl makes sense
    calculate_muscl = False

    # initialise flowlines
    if used_bed_shape == 'rectangular':
        oggm_fl = oggm_rect_flowline(surface_h=geometry['bed_h'],
                                     bed_h=geometry['bed_h'],
                                     widths=geometry['widths']/geometry['map_dx'],
                                     map_dx=geometry['map_dx'])


        combine_fl = combine_rect_flowline(surface_h=geometry['bed_h'],
                                           bed_h=torch.tensor(geometry['bed_h'],
                                                              dtype=torch.double,
                                                              requires_grad=False),
                                           widths_m=torch.tensor(geometry['widths'],
                                                              dtype=torch.double,
                                                              requires_grad=False),
                                           map_dx=geometry['map_dx'],
                                           torch_type=torch.double)

        if used_widths == 'constant':
            calculate_muscl = True

    elif used_bed_shape == 'parabolic':
        oggm_fl = oggm_para_flowline(surface_h=geometry['bed_h'],
                                     bed_h=geometry['bed_h'],
                                     bed_shape=geometry['bed_shape'],
                                     map_dx=geometry['map_dx'])

        combine_fl = combine_para_flowline(surface_h=geometry['bed_h'],
                                           bed_h=torch.tensor(geometry['bed_h'],
                                                              dtype=torch.double,
                                                              requires_grad=False),
                                           bed_shape=torch.tensor(geometry['bed_shape'],
                                                              dtype=torch.double,
                                                              requires_grad=False),
                                           map_dx=geometry['map_dx'],
                                           torch_type=torch.double)

    elif used_bed_shape == 'trapezoidal':
        oggm_fl = oggm_trap_flowline(surface_h=geometry['bed_h'],
                                     bed_h=geometry['bed_h'],
                                     widths=geometry['w0']/geometry['map_dx'],
                                     map_dx=geometry['map_dx'],
                                     lambdas=1)

        lambdas = torch.tensor([1.],
                               dtype=torch.double,
                               requires_grad=False)
        combine_fl = combine_trap_flowline(surface_h=geometry['bed_h'],
                                           bed_h=torch.tensor(geometry['bed_h'],
                                                              dtype=torch.double,
                                                              requires_grad=False),
                                           w0_m=torch.tensor(geometry['w0'],
                                                              dtype=torch.double,
                                                              requires_grad=False),
                                           map_dx=geometry['map_dx'],
                                           lambdas=lambdas,
                                           torch_type=torch.double)

    # define mass blance models
    combine_mb_model = define_mb_model(mb_opts={'ELA': np.array([3100., 3300.]),
                                                'grad': np.array([4., 4.])})

    oggm_mb_model = [oggm_MassBalance(
                            to_numpy_array(combine_mb_model[0].ela_h),
                            grad=to_numpy_array(combine_mb_model[0].grad)),
                         oggm_MassBalance(
                             to_numpy_array(combine_mb_model[1].ela_h),
                             grad=to_numpy_array(combine_mb_model[1].grad))]

    # make model runs and use oggm to define years of model runs
    if used_glacier_state == 'equilibrium':
        yrs_to_run = None

        oggm_model = oggm_FluxModel(oggm_fl, mb_model=oggm_mb_model[1], y0=0.)
        oggm_model.run_until_equilibrium()

        yrs_to_run = oggm_model.yr

        combine_model = combine_FluxModel(combine_fl, mb_model=combine_mb_model[1], y0=0.)
        combine_model.run_until(yrs_to_run)

        if calculate_muscl:
            muscl_model = MUSCLSuperBeeModel(oggm_fl, mb_model=oggm_mb_model[1], y0=0.)
            muscl_model.run_until(yrs_to_run)


    elif used_glacier_state == 'advancing':
        yrs_equilibrium = None
        years_to_advance = None
        for i in range(2):
            start_model = oggm_FluxModel(oggm_fl,
                                         mb_model=oggm_mb_model[1],
                                         y0=0.)
            start_model.run_until_equilibrium()
            yrs_equilibrium = start_model.yr
            oggm_model = oggm_FluxModel(start_model.fls[-1],
                                        mb_model=oggm_mb_model[0],
                                        y0=0.)
            if years_to_advance is None:
                oggm_model.run_until_equilibrium()
                years_to_advance = int(oggm_model.yr / 2)
            else:
                oggm_model.run_until(years_to_advance)
                break

        start_model = combine_FluxModel(combine_fl, mb_model=combine_mb_model[1], y0=0.)
        start_model.run_until(yrs_equilibrium)
        combine_model = combine_FluxModel(start_model.fls[-1], mb_model=combine_mb_model[0], y0=0.)
        combine_model.run_until(years_to_advance)

        if calculate_muscl:
            start_model = MUSCLSuperBeeModel(oggm_fl, mb_model=oggm_mb_model[1], y0=0.)
            start_model.run_until(yrs_to_run)
            muscl_model = MUSCLSuperBeeModel(start_model.fls[-1], mb_model=oggm_mb_model[0], y0=0.)
            muscl_model.run_until(years_to_advance)

    elif used_glacier_state == 'retreating':
        yrs_equilibrium = None
        years_to_retreat = None
        for i in range(2):
            start_model = oggm_FluxModel(oggm_fl,
                                         mb_model=oggm_mb_model[0],
                                         y0=0.)
            start_model.run_until_equilibrium()
            yrs_equilibrium = start_model.yr
            oggm_model = oggm_FluxModel(start_model.fls[-1],
                                        mb_model=oggm_mb_model[1],
                                        y0=0.)
            if years_to_retreat is None:
                oggm_model.run_until_equilibrium()
                years_to_retreat = int(oggm_model.yr / 2)
            else:
                oggm_model.run_until(years_to_retreat)
                break

        combine_model = combine_FluxModel(combine_fl, mb_model=combine_mb_model[0], y0=0.)
        combine_model.run_until(yrs_equilibrium)
        combine_model = combine_FluxModel(combine_model.fls[-1], mb_model=combine_mb_model[1], y0=0.)
        combine_model.run_until(years_to_retreat)

        if calculate_muscl:
            start_model = MUSCLSuperBeeModel(oggm_fl, mb_model=oggm_mb_model[0], y0=0.)
            start_model.run_until(yrs_to_run)
            muscl_model = MUSCLSuperBeeModel(start_model.fls[-1], mb_model=oggm_mb_model[1], y0=0.)
            muscl_model.run_until(years_to_retreat)

    else:
        raise ValueError('Unknown glacier state!')

    # save results dictionary
    for model in different_models:
        if model == 'OGGM':
            results[used_bed_shape][used_bed_h][used_widths][used_glacier_state][model]['surface_h'] = oggm_model.fls[0].surface_h
            results[used_bed_shape][used_bed_h][used_widths][used_glacier_state][model]['volume'] = oggm_model.fls[0].volume_km3
        elif model == 'COMBINE':
            results[used_bed_shape][used_bed_h][used_widths][used_glacier_state][model]['surface_h'] = to_numpy_array(combine_model.fls[0].surface_h)
            results[used_bed_shape][used_bed_h][used_widths][used_glacier_state][model]['volume'] = to_numpy_array(combine_model.fls[0].volume_km3)
        elif (model == 'MUSCL') and calculate_muscl:
            results[used_bed_shape][used_bed_h][used_widths][used_glacier_state][model]['surface_h'] = muscl_model.fls[0].surface_h
            results[used_bed_shape][used_bed_h][used_widths][used_glacier_state][model]['volume'] = muscl_model.fls[0].volume_km3

# save results

In [ ]:
import pickle

with open('model_comparison.pickle', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

pet_stats = {'name':"a", 'int':1, 'bool':False}

def pet_save(pet):
    with open(pet['name'] + '.pickle', 'wb') as f:
        pickle.dump(pet, f, pickle.HIGHEST_PROTOCOL)

def digimon_load(pet_name):
    with open(pet_name + '.pickle', 'rb') as f:
        return pickle.load(f)

In [ ]:
results